In [1]:
import numpy as np
import polars as pl
import pandas as pd
from data_prep_utilities import *
from dataset_descriptions import dataset_full, dataset_example

# Data Preparation Notebook

This notebook loads the data, performs feature selection and engineering, and joins the tables. The end result is a Train/Val/Test split, to be used for any model training.

## Data Explanation

A couple notes on data interpretation:

Where predictors were transformed, columns describing the transformation have been added with a capital letter suffixing the predictor name
* P - Transform DPD (Days past due)
* M - Masking categories
* A - Transform amount
* D - Transform date
* T - Unspecified Transform
* L - Unspecified Transform

On depths: depth of a table refers to how many num_group# columns are used to index. Each case_id is only featured once for each unique set of indices, although it may not have a listing for every set. The indexing is not necessarily chronological either; dates where num_group1 == 2 may be earlier than dates where num_group1 == 0. It may be useful to pull summary information for each case_id, e.g. min, max, median, fraction_empty.

In [2]:
# for exploration purposes: this gives more information about each feature
dataPath = "/kaggle/input/home-credit-credit-risk-model-stability/"
feature_definitions = pl.read_csv(dataPath + "feature_definitions.csv")
print(feature_definitions.head())

shape: (5, 2)
┌─────────────────────────┬───────────────────────────────────┐
│ Variable                ┆ Description                       │
│ ---                     ┆ ---                               │
│ str                     ┆ str                               │
╞═════════════════════════╪═══════════════════════════════════╡
│ actualdpd_943P          ┆ Days Past Due (DPD) of previous … │
│ actualdpdtolerance_344P ┆ DPD of client with tolerance.     │
│ addres_district_368M    ┆ District of the person's address… │
│ addres_role_871L        ┆ Role of person's address.         │
│ addres_zip_823M         ┆ Zip code of the address.          │
└─────────────────────────┴───────────────────────────────────┘


In [12]:
# for exploration: investigate a particular df or set of dfs
df_info = {
    "name":"tax_registry_c_1",
    "depth":2,
    "feature_types":["A","M"]
}
train_df, submit_df = load_df(**df_info)
train_df.head()

case_id,max_employername_160M,max_pmtamount_36A
i64,str,f64
127446,"""ec45a5ff""",2199.8
687087,"""d043a1df""",4682.59
1476377,"""e6f36ea5""",850.0
1317009,"""b0572451""",4363.2
12273,"""01fa175d""",700.0


In [13]:
# create a generator to step through features and their descriptions
cols=train_df.columns
if df_info['depth'] > 0:
    cols = [c[4:] for c in cols]
pl.Config.set_tbl_width_chars(100)
desc = feature_definitions.filter(pl.col('Variable').is_in(cols)).rows()
def next_row(desc):
    for row in desc:
        print(row[0],":")
        print(row[1])
        yield
row = next_row(desc)
print(len(desc))

2


In [7]:
next(row)

employername_160M :
Employer's name.


# Load Data

## Example: Generating splits from dataset descriptions

Below is a small dataset description; in fact, it describes the same dataset used in the starter notebook.

In [8]:
####################################################
# stores dataset info, arguments for load_df
#    description: notes to self. Ignored by load functions
#    name: from the actual name of the file, ignoring extra info (e.g., train/train_{NAME}_1.csv)
#    features (default all): specify columns to keep (ignore all others)
#    feature_types (default all): from kept features, select only those ending with these tags
#    depth (default 0): from kaggle description. If >0, aggregation will be performed
#    aggs (default ["agg_max"]): which aggregations to use (from agg_max, agg_min, agg_median)
#####################################################
dataset_example = {
    "base":{
        "description": "links case_id to WEEK_NUM and target",
        "name":"base",
    },
    "static_0":{
        "description":"contains transaction history for each case_id (late payments, total debt, etc)",
        "name":"static_0",
        "feature_types":["A", "M"],
    },
    "static_cb":{
        "description":"data from an external cb: demographic data, risk assessment, number of credit checks",
        "name":"static_cb",
        "feature_types":["A", "M"],
    },
    "person_1_feats_1":{
        "description":" internal demographic information: zip code, marital status, gender etc (all hashed)",
        "name":"person_1",
        "features":["mainoccupationinc_384A", "incometype_1044T"],
        "depth":1,
    },
    "person_1_feats_2":{
        "description":" internal demographic information: zip code, marital status, gender etc (all hashed)",
        "name":"person_1",
        "features":["housetype_905L"],
        "depth":1,
    },
    "credit_bureau_b_2":{
        "description":"historical data from an external source, num and value of overdue payments",
        "name":"credit_bureau_b_2",
        "features":["pmts_pmtsoverdue_635A","pmts_dpdvalue_108P"],
        "depth":2,
    }
}

We call load_all_dfs to load the specified datasets from csv, select features, aggregate as indicated, then join all.

In [14]:
train_df, submission_df = load_all_dfs(dataset_example)

We will only use submission_df at the end. We save our model's results on this submission_df data for kaggle to evaluate. Train_df is passed to our split function, which returns the splits ready for scaling and training. We also pass in the submission_df to standardize format.

In [15]:
train_sets, val_sets, test_sets, submit_sets = train_val_test_split(train_df, submission_df, train_split=0.6)

These sets are lists of pandas dfs of the form 
* base (case_id, WEEK_NUM, target)
* X (all predictor columns)
* y (target only)

In [16]:
train_sets[1].shape # X_train

(915995, 53)

In [17]:
train_sets[1].head()

,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,credamount_770A,currdebt_22A,currdebtcredtyperange_828A,...,maritalst_893M,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtssum_45A,max_mainoccupationinc_384A,max_incometype_1044T,max_housetype_905L,max_pmts_pmtsoverdue_635A,max_pmts_dpdvalue_108P
0,NaN,1917.6,0.0,NaN,NaN,NaN,NaN,30000.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,10800.0,SALARIED_GOVT,NaN,NaN,NaN
1,NaN,4937.0,0.0,NaN,NaN,NaN,NaN,78000.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,14000.0,EMPLOYED,NaN,NaN,NaN
2,NaN,3600.0,0.0,NaN,NaN,NaN,NaN,60000.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,64000.0,PRIVATE_SECTOR_EMPLOYEE,NaN,NaN,NaN
3,NaN,3110.8,0.0,NaN,NaN,NaN,NaN,20000.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,20000.0,EMPLOYED,NaN,NaN,NaN
4,NaN,1218.0,0.0,NaN,NaN,NaN,NaN,20300.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,46000.0,SALARIED_GOVT,NaN,NaN,NaN


We can now use these splits to train a model. Note that depending on the model, there may still be imputation/scaling/other augmentation necessary.

Next, convert categorical columns for all to dummy variables:

In [18]:
max_categories = 5 
# there will be two additional dummies for each categorical: "Unknown" and "nan"
# If there are more categories than our max, the least common items will be compressed into "Unknown"

# we must perform all dummy creation at the same time.
# if values in X_test are not found in X_train, they need to be marked 'Unknown' even if they are most frequent in X_test
X_train, X_val, X_test, X_submit = cat_to_dummies(train_sets[1], 
                                                  other_dfs = [val_sets[1], test_sets[1], submit_sets[1]], 
                                                  max_categories=max_categories)

In [19]:
X_train.head()

,lastapprcommoditycat_1041M_P109_133_183,lastapprcommoditycat_1041M_P12_6_178,lastapprcommoditycat_1041M_P148_110_5,lastapprcommoditycat_1041M_P159_130_59,lastapprcommoditycat_1041M_Unknown,lastapprcommoditycat_1041M_a55475b1,lastapprcommoditycat_1041M_nan,lastapprcommoditytypec_5251766M_P111_89_135,lastapprcommoditytypec_5251766M_P142_50_170,lastapprcommoditytypec_5251766M_P174_113_42,...,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtssum_45A,max_mainoccupationinc_384A,max_pmts_pmtsoverdue_635A,max_pmts_dpdvalue_108P
0,False,False,False,False,False,True,False,False,False,False,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10000.0,NaN,NaN
1,False,False,False,False,False,True,False,False,False,False,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10000.0,NaN,NaN
2,False,False,False,False,False,True,False,False,False,False,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,100000.0,NaN,NaN
3,False,False,False,False,False,True,False,False,False,False,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,50000.0,NaN,NaN
4,False,False,False,False,False,True,False,False,False,False,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,64000.0,NaN,NaN
